# 1.0 | Imports


## 1.1.0 | Import libraries

In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import requests
import nltk
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import time
import cudf
import dask_cudf

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:

## 1.2.0 | Import dataset

In [2]:
# Download latest version
path = kagglehub.dataset_download("carlosgdcj/genius-song-lyrics-with-language-information")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/genius-song-lyrics-with-language-information


In [4]:
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filepath = os.path.join(path, filename)
        try:
            # df = pd.read_csv(filepath, index_col=False)
            df = dask_cudf.read_csv(filepath) # Requires use of GPU

            print(f"Successfully loaded {filename} into a DataFrame.")
            break  # Assuming only one CSV file in the dataset
        except pd.errors.ParserError:
            print(f"Error parsing {filename}. Skipping this file.")
        except Exception as e:
            print(f"An error occurred while loading {filename}: {e}")

An error occurred while loading song_lyrics.csv: An error occurred while calling the read_csv method registered to the cudf backend.
Original Message: std::bad_alloc: CUDA error at: /pyenv/versions/3.12.9/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorInsufficientDriver CUDA driver version is insufficient for CUDA runtime version


In [5]:
print(df.dtypes)


title            object
tag              object
artist           object
year              int64
views             int64
features         object
lyrics           object
id                int64
language_cld3    object
language_ft      object
language         object
dtype: object


In [6]:
df.head(5)

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [7]:
# Print some info
print(df.info())
df.head(5)

<class 'dask_cudf._expr.collection.DataFrame'>
Columns: 11 entries, title to language
dtypes: object(8), int64(3)None


,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


# 2.0 | Preprocessing

## 2.1.0 | Remove unnecessary features

In [8]:
df.columns

Index(['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics', 'id',
       'language_cld3', 'language_ft', 'language'],
      dtype='object')

In [9]:
columns_to_drop = ['year', 'artist' 'views', 'features', 'id', 'language_cld3', 'language_ft']
df = df.drop(columns=columns_to_drop, errors='ignore')




In [10]:
df.head(5)

,title,tag,artist,views,lyrics,language
0,Killa Cam,rap,Cam'ron,173166,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",en
1,Can I Live,rap,JAY-Z,468624,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",en
2,Forgive Me Father,rap,Fabolous,4743,Maybe cause I'm eatin\nAnd these bastards fien...,en
3,Down and Out,rap,Cam'ron,144404,[Produced by Kanye West and Brian Miller]\n\n[...,en
4,Fly In,rap,Lil Wayne,78271,"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",en


In [11]:
# Drop rows where the language is not 'en'
df = df[df['language'] == 'en']


## 2.2.0 | Dataset Sampling

In [12]:
print(df.npartitions)

34


In [13]:
df_sampled = df.sample(frac=0.3)  # Keeps 30% of the dataset

In [26]:
print(df_sampled.npartitions)

34


In [14]:
# Convert columns to consistent types
for col in df.columns:
    df_sampled[col] = df[col].astype("object")  # or another common dtype


In [15]:
df_cudf = df_sampled.compute()

In [16]:
df_cudf.shape

(1012259, 6)

In [17]:
print(df_cudf.info())
df_cudf.head(5)

<class 'cudf.core.dataframe.DataFrame'>
Index: 1012259 entries, 22200 to 97521
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   title     1012230 non-null  object
 1   tag       1012259 non-null  object
 2   artist    1012259 non-null  object
 3   views     1012259 non-null  object
 4   lyrics    1012259 non-null  object
 5   language  1012259 non-null  object
dtypes: object(6)
memory usage: 1.6+ GB
None


,title,tag,artist,views,lyrics,language
22200,Bitches,rap,Insane Clown Posse,17748,[Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...,en
19066,The Master,rap,Flesh-N-Bone,493,[Intro]\nCome now say come now challenge the m...,en
34780,The Double K Show,rap,People Under the Stairs,864,"[Thes One]\nIt's not turntablism, it's hip hop...",en
33048,Holiday,rap,Naughty By Nature,6376,"[Chorus: Phiness]\nWe came here to party, join...",en
61377,Grand Scam Lyrical Exercise,rap,Azealia Banks,11107,[Intro]\nThese bitches be about a little get a...,en


In [18]:
df_pandas = df_cudf.to_pandas()


In [19]:
df_pandas = df_pandas.reset_index(drop=True)
print(type(df_pandas))
print(df_pandas.head())


<class 'pandas.core.frame.DataFrame'>
                         title  tag                   artist  views  \
0                      Bitches  rap       Insane Clown Posse  17748   
1                   The Master  rap             Flesh-N-Bone    493   
2            The Double K Show  rap  People Under the Stairs    864   
3                      Holiday  rap        Naughty By Nature   6376   
4  Grand Scam Lyrical Exercise  rap            Azealia Banks  11107   

                                              lyrics language  
0  [Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...       en  
1  [Intro]\nCome now say come now challenge the m...       en  
2  [Thes One]\nIt's not turntablism, it's hip hop...       en  
3  [Chorus: Phiness]\nWe came here to party, join...       en  
4  [Intro]\nThese bitches be about a little get a...       en  


In [20]:
# Drops residual columns
columns_to_drop = ['artist', 'views', 'language', 'title']
df_cleaned = df_pandas.drop(columns=columns_to_drop, errors='ignore')


## 2.2.0 | Clean Song Lyrics

In [21]:
df2 = df_cleaned.copy()
# df2 = df2.iloc[:20000]
df2.head(5)

,tag,lyrics
0,rap,[Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...
1,rap,[Intro]\nCome now say come now challenge the m...
2,rap,"[Thes One]\nIt's not turntablism, it's hip hop..."
3,rap,"[Chorus: Phiness]\nWe came here to party, join..."
4,rap,[Intro]\nThese bitches be about a little get a...


In [22]:
df2.shape

(1012259, 2)

In [23]:
stop_words = set(stopwords.words('english'))

stop_words.update(["yeah", "nah", "da", "uh", "um", "yo", "huh", "mhm", "ok", "okay"])  # Add filler words
stop_words.update(["like", "know", "think", "feel", "want", "need", "make", "get", "go", "come"])  # Add common words
stop_words.update(["cannot", "would", "could", "should", "will", "not",  "is", "are", "am", "have", "has", "had"]) # Helps remove contractions
stop_words.update(["i", "me", "my", "mine", "you", "your", "yours", "he", "him", "his", "she", "her", "hers", "it", "its", "we", "us", "our", "ours", "they", "them", "their", "theirs"]) #adds pronouns
stop_words.update(["in", "on", "at", "by", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below"]) # Add prepositions
stop_words.update(["and", "but", "or", "nor", "for", "yet", "so"]) # Add conjunctions
stop_words.update(["must", "shall", "will", "should", "would", "can", "could", "may", "might", "ought"]) # Adds modal verbs
stop_words.update(["yeah", "nah", "da", "uh", "um", "yo", "huh", "mhm", "ok", "okay"])  # Add filler words
stop_words.update(["baby", "honey", "gonna", "wanna", "gotta", "chorus", "verse", "boom", "bang"]) # Add new words
# General Filler and Common Words
stop_words.update(["oh", "ah", "ooh", "yeah", "hey", "whoa", "nah", "ayy", "yo", "huh"])  # Interjections and Exclamations
stop_words.update(["baby", "honey", "darling", "lover", "sweetheart", "mama", "papa"])  # Repetitive Phrases
stop_words.update(["gonna", "wanna", "gotta", "ya", "bruh", "bro", "dude", "girl", "boy", "man"])  # Slang and Colloquialisms
stop_words.update(["said", "see", "saw", "got", "went", "came", "know", "say", "tell", "give", "take", "put"])  # Common Verbs
stop_words.update(["things", "stuff", "something", "anything", "everything", "nothing", "somebody", "anybody", "everybody", "nobody", "somewhere", "anywhere", "everywhere", "nowhere"])  # Other
stop_words.update(["chorus", "verse", "intro", "bridge", "outro", "hook", "pre-chorus"])  # Chorus/Verse Indicators
stop_words.update(["boom", "bang", "pow", "bam", "clap", "snap", "woosh", "ding", "buzz"])

def remove_stopwords(text):
    if pd.isna(text):
        return ''
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [24]:
def clean_lyrics(text):
    if pd.isna(text):  # Handle missing values
        return ''

    # Removes contractions
    text = re.sub(r"won't", "will not", text)

    text = re.sub(r"can\'t", "cannot", text)

    text = re.sub(r"n\'t", " not", text)

    text = re.sub(r"\'re", " are", text)

    text = re.sub(r"\'s", " is", text)

    text = re.sub(r"\'d", " would", text)

    text = re.sub(r"\'ll", " will", text)

    text = re.sub(r"\'t", " not", text)

    text = re.sub(r"\'ve", " have", text)

    text = re.sub(r"\'m", " am", text)

    text = text.lower() # Convert to lowercase

    text = re.sub(r'[^\x00-\x7F]+', ' ', text) # Remove non-ASCII characters

    text = re.sub(r'\s*\'\s*', '', text) # Remove apostrophes and merge contractions

    text = re.sub(r'[^a-z0-9\s]', ' ', text) # Remove all punctuation except spaces, keep letters and numbers

    text = re.sub(r'\d+embed$', '', text) # Remove number followed by "embed" at the end

    text = re.sub(r'\s+', ' ', text).strip() # Replace multiple spaces with a single space and remove leading/trailing whitespace

    text = re.sub(r'\b\w\b', '', text) # Remove single letters

    text = re.sub(r'\b\d+\b', '', text) # Remove numbers

    text = re.sub(r'\b\w{1,2}\b', '', text) # Remove words with 1 or 2 letters

    text = re.sub(r"\[.*?\]", "", text)

    text = remove_stopwords(text) # Removes stop words


    return text

In [25]:
# Cleans the lyrics of the dataset
df2['cleaned_lyrics'] = df2['lyrics'].apply(clean_lyrics)

In [27]:
df2.head(5)

,tag,lyrics,cleaned_lyrics
0,rap,[Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...,oldirty bastard shaggy dope bitches bitches bi...
1,rap,[Intro]\nCome now say come now challenge the m...,challenge master caught disaster blastem dead ...
2,rap,"[Thes One]\nIt's not turntablism, it's hip hop...",thes one turntablism hip hop turntablism hip h...
3,rap,"[Chorus: Phiness]\nWe came here to party, join...",phiness party join together let celebrate part...
4,rap,[Intro]\nThese bitches be about a little get a...,bitches little grand scam blam nigga bet nigga...


# 3.0 | Viewing Genre Data

In [29]:
df3 = df2.copy()
df3.head(5)


,tag,lyrics,cleaned_lyrics
0,rap,[Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...,oldirty bastard shaggy dope bitches bitches bi...
1,rap,[Intro]\nCome now say come now challenge the m...,challenge master caught disaster blastem dead ...
2,rap,"[Thes One]\nIt's not turntablism, it's hip hop...",thes one turntablism hip hop turntablism hip h...
3,rap,"[Chorus: Phiness]\nWe came here to party, join...",phiness party join together let celebrate part...
4,rap,[Intro]\nThese bitches be about a little get a...,bitches little grand scam blam nigga bet nigga...


## 3.1.0 | View Genres

In [9]:
unique_tags = df3['tag'].unique()
unique_tags


array(['rap', 'misc', 'pop', 'rock', 'rb', 'country'], dtype=object)

In [32]:
df3.shape

(1012259, 3)

# 4.0 | Top 100 words

## 4.1.0 | Lemmatize dataset

In [33]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(lemmatized_words)

df3['lemmatized_lyrics'] = df2['cleaned_lyrics'].apply(lemmatize_text)
df3.head(5)


,tag,lyrics,cleaned_lyrics,lemmatized_lyrics
0,rap,[Intro: Ol' Dirty Bastard & Shaggy 2 Dope]\nBi...,oldirty bastard shaggy dope bitches bitches bi...,oldirty bastard shaggy dope bitch bitch bitch ...
1,rap,[Intro]\nCome now say come now challenge the m...,challenge master caught disaster blastem dead ...,challenge master caught disaster blastem dead ...
2,rap,"[Thes One]\nIt's not turntablism, it's hip hop...",thes one turntablism hip hop turntablism hip h...,thes one turntablism hip hop turntablism hip h...
3,rap,"[Chorus: Phiness]\nWe came here to party, join...",phiness party join together let celebrate part...,phiness party join together let celebrate part...
4,rap,[Intro]\nThese bitches be about a little get a...,bitches little grand scam blam nigga bet nigga...,bitch little grand scam blam nigga bet nigga s...


In [39]:
df3.to_csv('df3.csv', index=False)


In [ ]:
df3 = pd.read_csv('df3.csv')

## 4.2.0 | Top 100 words

In [41]:
def extract_top_words(df, parent_genre, top_n=100):
    # Filter the DataFrame for the specified parent genre
    genre_df = df[df['tag'] == parent_genre]

    # Combine all lyrics into a single string
    all_lyrics = ' '.join(genre_df['lemmatized_lyrics'].astype(str))

    # Tokenize the lyrics
    words = word_tokenize(all_lyrics)

    # Count word frequencies
    word_counts = Counter(words)

    # Get the top N most frequent words
    # top_words = word_counts.most_common(top_n)
    top_words = [word for word, _ in word_counts.most_common(top_n)]

    return top_words

# Create a list to store the results for all genres
all_genres_top_words = []

for genre in unique_tags:
    top_words_for_genre = extract_top_words(df3, genre)
    all_genres_top_words.extend([[genre, word] for word in top_words_for_genre])
    print(f"{genre},{top_words_for_genre}")

print(all_genres_top_words)

rap,['shit', 'bitch', 'fuck', 'nigga', 'back', 'never', 'niggas', 'love', 'one', 'time', 'let', 'cause', 'life', 'way', 'money', 'keep', 'right', 'still', 'really', 'look', 'hit', 'even', 'real', 'tryna', 'day', 'call', 'better', 'fucking', 'new', 'every', 'always', 'god', 'mind', 'good', 'ass', 'big', 'stay', 'run', 'around', 'people', 'bitches', 'stop', 'bad', 'made', 'head', 'getting', 'going', 'hard', 'face', 'told', 'night', 'talk', 'leave', 'play', 'ever', 'world', 'show', 'try', 'game', 'gone', 'pull', 'damn', 'yall', 'high', 'heart', 'live', 'two', 'away', 'done', 'feeling', 'much', 'lil', 'little', 'die', 'beat', 'top', 'name', 'watch', 'aye', 'bout', 'yea', 'hate', 'hold', 'young', 'turn', 'black', 'find', 'whole', 'work', 'best', 'rap', 'ride', 'ama', 'long', 'thing', 'left', 'another', 'eyes', 'talking', 'cuz']
misc,['one', 'time', 'well', 'good', 'people', 'never', 'back', 'day', 'way', 'little', 'upon', 'made', 'two', 'life', 'first', 'let', 'love', 'even', 'much', 'great

In [43]:
# Create a Pandas DataFrame from the list of top words
top_words_df = pd.DataFrame(all_genres_top_words, columns=['Genre', 'Word'])

# Save the DataFrame to a CSV file
top_words_df.to_csv('top_words.csv', index=False)